## Initialization and Loading Point Clouds
- Import required functions and modules from the vapc library.
- Set paths for the two point cloud files along with output directory paths.
- Define key parameters such as the voxel size (6 meters) and the alpha value (0.999) for later computations.
- Optionally disable tracing and timing functionality to keep the output clean.
- Convert the two point clouds to their vapc representations using `point_cloud_to_vapc`.

In [ ]:
import vapc
from vapc.vapc_tools import point_cloud_to_vapc, extract_point_cloud_by_3D_mask
import os

# Optional: Silent mode
vapc.enable_trace(False)
vapc.enable_timeit(False)

point_cloud_1_path = r"path_to_point_cloud_1.laz"
point_cloud_2_path = r"path_to_point_cloud_2.laz"

output_folder = r"output_folder"
output_file_full_voxelized = os.path.join(output_folder, "full_voxelized.laz")
output_file_changed_voxelized = os.path.join(output_folder, "changes_voxelized.laz")
voxel_size = 6 #m
alpha_value = 0.999 

vapc_1 = point_cloud_to_vapc(point_cloud_file=point_cloud_1_path,
                           voxel_size=voxel_size)

vapc_2 = point_cloud_to_vapc(point_cloud_file=point_cloud_2_path,
                           voxel_size=voxel_size)

- A BiTemporalVapc instance is created using the two vapc representations (vapc_1 and vapc_2). This object is designed to handle and compare datasets from different time epochs.
- The method prepare_data_for_mahalanobis_distance() processes and aligns the data, setting up the features required for the Mahalanobis distance calculation.
- The function merge_vapcs_with_same_voxel_index() consolidates voxels from both point clouds that have the same spatial location, ensuring that the subsequent comparison happens voxel-by-voxel.
- The compute_distance() method calculates the Euclidean distance between corresponding voxels in the two datasets.
- The compute_mahalanobis_distance(alpha=alpha_value) method then computes the Mahalanobis distance for these voxels, using an alpha value as a threshold based on the chi-squared distribution. This helps in identifying statistically significant differences.
- Finally, compute_voxels_occupied_in_single_epoch() identifies voxels that appear in only one of the point clouds, marking regions where changes such as points disappearing or appearing are observed.

In [3]:
bi_vapc = vapc.BiTemporalVapc([vapc_1, vapc_2])
bi_vapc.prepare_data_for_mahalanobis_distance() #prepare data for mahalanobis distance
bi_vapc.merge_vapcs_with_same_voxel_index() #defines how comparison is done. Here, it is done per same voxel index.
bi_vapc.compute_distance() #euclidean distance
bi_vapc.compute_mahalanobis_distance(alpha=alpha_value) #alpha value for chi2
bi_vapc.compute_voxels_occupied_in_single_epoch() #disappearing and appearing voxels

- Prepare data for export
- Save the areas where changes have been detected

In [4]:
# Prepare data for export
bi_vapc.prepare_data_for_export()
# Optional saves the full voxelized point cloud with mahalanobis distance, euclidean distance, and change type.
# bi_vapc.save_to_las(output_file_full_voxelized) 

# But we only want areas where change might have happened. So areas where the change 
# type is 1 (mahalanobis significant), 2 (less than 30 points in voxel), 3 (disappearing), 
# and 4 (appearing) are kept.
bi_vapc.df = bi_vapc.df[(bi_vapc.df["change_type"] >= 1)]
bi_vapc.save_to_las(output_file_changed_voxelized)

- Extract areas with change from full point clouds

In [5]:
# Clip area from T1 and T2
t1_outfile = os.path.join(output_folder, "t1.laz")
t2_outfile = os.path.join(output_folder, "t2.laz")

extract_point_cloud_by_3D_mask(point_cloud_1_path, output_file_changed_voxelized, t1_outfile, voxel_size)
extract_point_cloud_by_3D_mask(point_cloud_2_path, output_file_changed_voxelized, t2_outfile, voxel_size)